# 第4章: マルチエージェント・ワークフロー

## 準備

以下のセルを順番に実行して、演習に必要な環境をセットアップします。

### ライブラリのインストール

このセルは、LangGraphおよび関連するLangChainライブラリをインストールします。

In [ ]:
# === ライブラリのインストール ===
!pip install -qU langchain langgraph langchain_openai langchain_community duckduckgo-search

# --- その他の推奨ライブラリ ---
!pip install -qU python-dotenv pygraphviz pydotplus graphviz

### OpenAI APIキーの設定

このノートブックでは、エージェントの中核となるLLMとしてOpenAIのモデルを使用します。
事前にOpenAIのAPIキーを取得し、環境変数 `OPENAI_API_KEY` に設定するか、Google Colabの場合はColabのシークレットマネージャーに `OPENAI_API_KEY` という名前でキーを登録してください。

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

try:
    from google.colab import userdata
    if "OPENAI_API_KEY" in userdata.get_all():
        os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    print("OpenAI APIキーをColabシークレットからロードしました。")
except ImportError:
    if os.getenv("OPENAI_API_KEY"):
        print("OpenAI APIキーを.envファイルからロードしました。")
    else:
        print("OpenAI APIキーが見つかりません。環境変数に設定するか、Colabシークレットに登録してください。")

# LLMの準備
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print(f"LLM ({llm.model_name}) の準備ができました。")

---

## 問題001: 基本的な2エージェント会話（リサーチャーとライター）

### 課題
マルチエージェントシステムの最も基本的な形として、2つの異なる役割を持つエージェント（例: リサーチャーとライター）が協調してタスクをこなすワークフローを構築します。リサーチャーが情報を収集し、その結果をライターが受け取って文章を生成する、という流れをLangGraphで実装します。

*   **学習内容:** 複数のエージェント（実態はLLMをラップした関数やノード）を定義し、それらを順番に接続して情報を引き継がせる方法を学びます。状態(`State`)がエージェント間の情報共有のハブとなることを理解します。

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated, List, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

# --- 状態定義 (State) ---
class TwoAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    topic: str
    research_data: Optional[str]
    written_article: Optional[str]

# --- ノード定義 (エージェントの役割を担う関数) ---
def researcher_agent_node(state: TwoAgentState):
    topic = state["topic"]
    print(f"Researcher: トピック「{topic}」について調査中...")
    # ダミーの調査結果 (実際にはWeb検索ツールなどを使う)
    # research_info = search_tool.invoke(topic) 
    research_info = f"「{topic}」に関する調査結果: {topic}は非常に興味深いトピックで、多くの側面があります。"
    return {"research_data": research_info, "messages": [AIMessage(content=f"調査結果: {research_info}", name="Researcher")]}

def writer_agent_node(state: TwoAgentState):
    data = state["research_data"]
    topic = state["topic"]
    print(f"Writer: 調査データ「{data[:50]}...」に基づいて記事を執筆中 (トピック: {topic})...")
    # ダミーの記事生成 (実際にはLLMを使う)
    # article = llm.invoke(f"以下の調査結果を元に、「{topic}」に関する短い記事を書いてください。\n調査結果: {data}").content
    article = f"# {topic}についての考察\n\n{data} これを元に考えると、{topic}の将来は明るいと言えるでしょう。"
    return {"written_article": article, "messages": [AIMessage(content=article, name="Writer")]}

# --- グラフ構築 ---
workflow = StateGraph(TwoAgentState)
workflow.add_node("researcher", researcher_agent_node)
workflow.add_node("writer", writer_agent_node)

workflow.set_entry_point("researcher")
workflow.add_edge("researcher", "writer")
workflow.add_edge("writer", END)

graph = workflow.compile()

# --- 実行 --- 
initial_topic = "AIと未来の仕事"
inputs = {"messages": [HumanMessage(content=f"「{initial_topic}」について記事を書いてください。")], "topic": initial_topic}
for event in graph.stream(inputs, {"recursion_limit": 5}):
    print(event)

final_state = graph.invoke(inputs, {"recursion_limit": 5})
print(f"\n最終成果物 (記事):\n{final_state['written_article']}")
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated, List, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import Image, display

# --- 状態定義 (State) ---
class TwoAgentCollaborationState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages] # エージェント間のメッセージやログ
    original_request: str       # ユーザーからの最初の要求
    research_findings: Optional[str] # リサーチャーが見つけた情報
    draft_article: Optional[str]     # ライターが書いた記事のドラフト
    final_article: Optional[str]     # 最終的な記事

# --- ノード定義 (各エージェントの役割を担う関数) ---
def researcher_agent_node(state: TwoAgentCollaborationState):
    request = state["original_request"]
    print(f"\nResearcher Agent: リクエスト「{request}」に関する情報を調査します...")
    
    # ダミーの調査プロセス (実際にはWeb検索ツール、DBアクセスなどを行う)
    # findings = search_tool.invoke(request) # 例: search_toolが準備セルで定義済みと仮定
    findings = f"「{request}」に関する重要な情報として、ポイントA、ポイントB、そして注目すべきトレンドCが見つかりました。"
    print(f"  -> 調査結果: {findings}")
    
    return {"research_findings": findings, "messages": [AIMessage(content=f"調査結果報告: {findings}", name="Researcher")]}

def writer_agent_node(state: TwoAgentCollaborationState):
    findings = state.get("research_findings")
    request = state.get("original_request")
    if not findings:
        print("Writer Agent: 調査結果がありません。記事を作成できません。")
        return {"messages": [AIMessage(content="調査結果が不足しているため、記事を作成できませんでした。", name="Writer")]}
    
    print(f"\nWriter Agent: 調査結果「{findings[:50]}...」とリクエスト「{request}」に基づいて記事を執筆します...")
    
    # LLMを使って記事を生成
    writer_prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="あなたはプロのライターです。与えられた調査結果と元のリクエストに基づいて、読者にとって魅力的で分かりやすい記事を作成してください。"),
        HumanMessage(content=f"元のリクエスト: {request}\n\n調査結果:\n{findings}\n\n記事本文:")
    ])
    writer_chain = writer_prompt | llm # llmは準備セルで初期化済み
    article_content = writer_chain.invoke({}).content # invokeに空辞書でOK (プロンプト内で完結)
    print(f"  -> 生成された記事ドラフト: {article_content[:100]}...")
    
    # この例ではドラフトをそのまま最終記事とする
    return {"draft_article": article_content, "final_article": article_content, "messages": [AIMessage(content=article_content, name="Writer")]}

# --- グラフ構築 ---
workflow = StateGraph(TwoAgentCollaborationState)

# ノードを追加
workflow.add_node("researcher", researcher_agent_node)
workflow.add_node("writer", writer_agent_node)

# エッジを定義
workflow.set_entry_point("researcher") # リサーチャーから開始
workflow.add_edge("researcher", "writer")    # リサーチャーの結果をライターへ
workflow.add_edge("writer", END)            # ライターの処理が終わったら終了

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
user_task_request = "再生可能エネルギーの将来性について解説する記事を作成してください。"
print(f"\n--- 基本的な2エージェント会話テスト (リクエスト: '{user_task_request}') ---")

inputs = {
    "messages": [HumanMessage(content=f"タスク指示: {user_task_request}")],
    "original_request": user_task_request
}

print("\nストリーム出力:")
for i, event in enumerate(graph.stream(inputs, {"recursion_limit": 5})):
    print(f"Event {i+1}: {event}")

final_state_2agents = graph.invoke(inputs, {"recursion_limit": 5})
print("\n最終状態の確認:")
print(f"  元のリクエスト: {final_state_2agents['original_request']}")
print(f"  調査結果: {final_state_2agents['research_findings'][:70]}...")
print(f"  最終記事:\n{final_state_2agents['final_article']}")

assert final_state_2agents['research_findings'] is not None
assert final_state_2agents['final_article'] is not None
assert user_task_request in final_state_2agents['final_article'] # 記事内容がリクエストに関連していることを簡易チェック
print("\nアサーション成功！")
``````
</details>

## 問題002: エージェントスウォームの基本 - スーパーバイザーによるタスク割り振り

### 課題
複数の専門エージェントが存在する場合、それらのエージェントを統括し、タスクに応じて適切なエージェントに処理を割り振る「スーパーバイザー」の役割が重要になります。この問題では、スーパーバイザーLLMと複数のワーカーエージェント（例: リサーチャー、コーダー）を定義し、スーパーバイザーがユーザーの要求を解釈して、対応するワーカーエージェントにタスクを指示する基本的なエージェントスウォームの形を構築します。

*   **学習内容:** スーパーバイザー（LLMノード）が状態やメッセージに基づいて次に実行すべきワーカーエージェント（別のLLMノードまたはツールノード）を決定し、条件付きエッジで処理をルーティングする方法を学びます。エージェント間でタスク情報や結果をどのように受け渡すかを状態設計で考慮します。

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated, List, Optional, Literal
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

WorkerName = Literal["Researcher", "Coder", "GeneralResponder", "Supervisor", "END_TASK"]

# --- 状態定義 (State) ---
class SupervisorAgentSwarmState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    user_query: str
    next_worker: Optional[WorkerName] # 次に実行すべきワーカー名
    task_data: Optional[str]        # ワーカーへの具体的な指示やデータ
    final_response: Optional[str]

# --- ワーカーエージェントのダミー実装 ---
def researcher_worker_node(state: SupervisorAgentSwarmState):
    task = state["task_data"]
    print(f"Researcher Worker: 「{task}」について調査実行中...")
    result = f"「{task}」の調査結果: 詳細データXYZ。"
    return {"messages": [AIMessage(content=result, name="Researcher")], "next_worker": "Supervisor"}

def coder_worker_node(state: SupervisorAgentSwarmState):
    task = state["task_data"]
    print(f"Coder Worker: 「{task}」のコーディングタスク実行中...")
    code_snippet = f"def {task.replace(' ', '_')}_solution():\n    print('Implement {task} here')"
    return {"messages": [AIMessage(content=code_snippet, name="Coder")], "next_worker": "Supervisor"}

def general_responder_node(state: SupervisorAgentSwarmState):
    query = state["user_query"]
    print(f"General Responder: 「{query}」に直接応答します...")
    response = f"「{query}」ですね、承知いたしました。一般的な応答です。"
    return {"final_response": response, "messages": [AIMessage(content=response, name="GeneralResponder")], "next_worker": "END_TASK"}

# --- スーパーバイザーノード ---
def supervisor_node(state: SupervisorAgentSwarmState):
    print("\nSupervisor: 次の行動を決定します...")
    last_message = state["messages"][-1]
    user_q = state["user_query"]
    
    if isinstance(last_message, HumanMessage): # 初回またはユーザーからの追加指示
        if "調べて" in user_q or "調査" in user_q:
            next_w, task_d = "Researcher", user_q
        elif "コード" in user_q or "プログラム" in user_q:
            next_w, task_d = "Coder", user_q
        else:
            next_w, task_d = "GeneralResponder", user_q
    else: # ワーカーからの結果を受けて
        print(f"  -> ワーカー ({last_message.name}) からの結果を受信: {last_message.content[:50]}...")
        # ここでは単純にタスク完了として終了させる
        next_w, task_d = "END_TASK", "タスク完了"
        # 実際には、結果を評価して別のワーカーに渡したり、ユーザーに返したりする
        final_resp = f"スーパーバイザーより: 「{user_q}」に関するタスクがワーカー「{last_message.name}」によって処理されました。結果: {last_message.content}"
        return {"next_worker": next_w, "task_data": task_d, "final_response": final_resp, "messages": [AIMessage(content=final_resp, name="Supervisor")]}

    print(f"  -> 次のワーカー: {next_w}, タスク: {task_d}")
    return {"next_worker": next_w, "task_data": task_d, "messages": [AIMessage(content=f"{next_w}へタスク「{task_d}」を割り当てます", name="Supervisor")]}

# --- ルーター関数 ---
def task_router(state: SupervisorAgentSwarmState):
    next_w = state.get("next_worker")
    print(f"Task Router: 次のワーカーは '{next_w}'")
    if next_w == "Researcher": return "to_researcher"
    if next_w == "Coder": return "to_coder"
    if next_w == "GeneralResponder": return "to_general_responder"
    return END # END_TASK または不明な場合は終了

# --- グラフ構築 ---
workflow = StateGraph(SupervisorAgentSwarmState)
workflow.add_node("supervisor", supervisor_node)
workflow.add_node("researcher_worker", researcher_worker_node)
workflow.add_node("coder_worker", coder_worker_node)
workflow.add_node("general_worker", general_responder_node)

workflow.set_entry_point("supervisor")
workflow.add_conditional_edges(
    "supervisor", task_router,
    {"to_researcher": "researcher_worker", "to_coder": "coder_worker", 
     "to_general_responder": "general_worker", END: END}
)
# 各ワーカーは処理後Supervisorに戻る
workflow.add_edge("researcher_worker", "supervisor")
workflow.add_edge("coder_worker", "supervisor")
workflow.add_edge("general_worker", "supervisor") # GeneralResponderはEND_TASKをセットするので、routerで終了

graph = workflow.compile()

# --- 実行 --- 
queries = [
    "LangGraphの最新の変更点を調べてください。",
    "Pythonで簡単な電卓プログラムの雛形を書いてください。",
    "こんにちは、今日の気分はどうですか？"
]
for q in queries:
    print(f"\n--- スーパーバイザーテスト (クエリ: {q}) ---")
    inputs = {"messages": [HumanMessage(content=q)], "user_query": q}
    for event in graph.stream(inputs, {"recursion_limit": 5}): print(event)
    final_state = graph.invoke(inputs, {"recursion_limit": 5})
    print(f"最終応答: {final_state.get('final_response')}") 
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated, List, Optional, Literal
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import Image, display

# ワーカーの種類を定義 (Literal型で型安全性を高める)
WorkerType = Literal["Researcher", "Coder", "GeneralResponder", "Supervisor", "FINISH"]

# --- 状態定義 (State) ---
class AgentSwarmState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    user_request: str                # ユーザーの元のリクエスト
    assigned_worker: Optional[WorkerType] # 次にタスクを実行するワーカー名
    task_description: Optional[str]  # ワーカーへの具体的なタスク指示
    task_result: Optional[str]       # ワーカーからのタスク結果
    final_answer: Optional[str]      # スーパーバイザーが生成する最終回答

# --- ワーカーエージェントのダミー実装ノード ---
def researcher_worker_node(state: AgentSwarmState):
    task_desc = state.get("task_description", "不明な調査タスク")
    print(f"\nResearcher Worker: タスク「{task_desc}」の調査を実行します...")
    # ダミー調査結果
    result = f"「{task_desc}」に関する調査結果です。主要なポイントはA, B, Cです。"
    print(f"  -> 調査結果: {result}")
    return {"task_result": result, "messages": [AIMessage(content=result, name="Researcher")]}

def coder_worker_node(state: AgentSwarmState):
    task_desc = state.get("task_description", "不明なコーディングタスク")
    print(f"\nCoder Worker: タスク「{task_desc}」のコーディングを実行します...")
    # ダミーコードスニペット
    code = f"# {task_desc} のためのPythonコード\ndef {task_desc.replace(' ', '_')}_solution():\n    pass # 実装してください"
    print(f"  -> 生成コード:\n{code}")
    return {"task_result": code, "messages": [AIMessage(content=code, name="Coder")]}

def general_responder_worker_node(state: AgentSwarmState):
    task_desc = state.get("task_description", state.get("user_request", "一般的な応答"))
    print(f"\nGeneral Responder Worker: 「{task_desc}」に一般的な応答をします...")
    # ダミー応答
    response = f"「{task_desc}」についてですね。それは非常に興味深いご質問です。私がお答えできる範囲で...
    print(f"  -> 応答: {response}")
    return {"task_result": response, "messages": [AIMessage(content=response, name="GeneralResponder")]}

# --- スーパーバイザーノード ---
def supervisor_agent_node(state: AgentSwarmState):
    print(f"\nSupervisor Agent: 現在の状態を評価し、次のアクションを決定します...")
    current_messages = state["messages"]
    user_request_text = state["user_request"]
    last_message = current_messages[-1] if current_messages else None

    # 初回呼び出し (ユーザーからのリクエスト時)
    if isinstance(last_message, HumanMessage) or state.get("assigned_worker") is None:
        print(f"  -> ユーザーリクエスト「{user_request_text}」を処理します。")
        # LLMを使って適切なワーカーを選択する (ここでは簡易的なルールベース)
        if "調査" in user_request_text or "調べて" in user_request_text or "とは" in user_request_text:
            assigned_next_worker: WorkerType = "Researcher"
            task_for_worker = user_request_text
        elif "コード" in user_request_text or "プログラム" in user_request_text or "実装" in user_request_text:
            assigned_next_worker: WorkerType = "Coder"
            task_for_worker = user_request_text
        else:
            assigned_next_worker: WorkerType = "GeneralResponder"
            task_for_worker = user_request_text
        print(f"  -> 次のワーカー: {assigned_next_worker}, タスク指示: {task_for_worker}")
        return {"assigned_worker": assigned_next_worker, "task_description": task_for_worker, "messages": [AIMessage(content=f"スーパーバイザーより: {assigned_next_worker}に「{task_for_worker}」を指示します。", name="Supervisor")]}
    
    # ワーカーからの結果を受け取った場合
    elif isinstance(last_message, AIMessage) and last_message.name != "Supervisor":
        worker_name = last_message.name
        worker_result = state.get("task_result", "(結果なし)")
        print(f"  -> ワーカー「{worker_name}」から結果「{worker_result[:50]}...」を受信しました。")
        # このデモでは、ワーカーが一度仕事したらタスク完了とする
        final_answer_text = f"ユーザーリクエスト「{user_request_text}」に対し、ワーカー「{worker_name}」が以下の結果を報告しました：\n{worker_result}"
        print(f"  -> 最終回答を生成: {final_answer_text[:70]}...")
        return {"assigned_worker": "FINISH", "final_answer": final_answer_text, "messages": [AIMessage(content=final_answer_text, name="Supervisor")]}
    
    # 予期せぬ状況 (フォールバック)
    print("  -> 予期せぬ状態。GeneralResponderにフォールバックします。")
    return {"assigned_worker": "GeneralResponder", "task_description": user_request_text, "messages": [AIMessage(content="スーパーバイザーより: 予期せぬ状況のためGeneralResponderに対応を依頼します。", name="Supervisor")]}

# --- ルーター関数 ---
def supervisor_router(state: AgentSwarmState):
    next_worker_to_call = state.get("assigned_worker")
    print(f"\nSupervisor Router: 次に呼び出すワーカーは '{next_worker_to_call}' です。")
    if next_worker_to_call == "Researcher": return "route_to_researcher"
    if next_worker_to_call == "Coder": return "route_to_coder"
    if next_worker_to_call == "GeneralResponder": return "route_to_general_responder"
    if next_worker_to_call == "FINISH": return END
    return END # デフォルトは終了

# --- グラフ構築 ---
workflow = StateGraph(AgentSwarmState)

workflow.add_node("supervisor", supervisor_agent_node)
workflow.add_node("researcher_worker", researcher_worker_node)
workflow.add_node("coder_worker", coder_worker_node)
workflow.add_node("general_worker", general_responder_worker_node)

workflow.set_entry_point("supervisor")

workflow.add_conditional_edges(
    "supervisor", 
    supervisor_router,
    {
        "route_to_researcher": "researcher_worker", 
        "route_to_coder": "coder_worker", 
        "route_to_general_responder": "general_worker", 
        END: END
    }
)

# 各ワーカーは処理後、結果を持ってSupervisorに戻る
workflow.add_edge("researcher_worker", "supervisor")
workflow.add_edge("coder_worker", "supervisor")
workflow.add_edge("general_worker", "supervisor")

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
test_queries = [
    "LangChain Expression Language (LCEL) について調査してください。",
    "Pythonでフィボナッチ数列を計算する関数を実装してください。",
    "こんにちは、お元気ですか？"
]

for query in test_queries:
    print(f"\n--- スーパーバイザーによるタスク割り振りテスト (クエリ: '{query}') ---")
    inputs = {"messages": [HumanMessage(content=query)], "user_request": query}
    
    print("\nストリーム出力:")
    for i, event in enumerate(graph.stream(inputs, {"recursion_limit": 5})):
        print(f"Event {i+1}: {event}")
    
    final_state_swarm = graph.invoke(inputs, {"recursion_limit": 5})
    print("\n最終状態の確認:")
    print(f"  ユーザーリクエスト: {final_state_swarm['user_request']}")
    print(f"  最終割り当てワーカー(ルーター判断前): {final_state_swarm['assigned_worker']}")
    print(f"  タスク結果: {final_state_swarm.get('task_result', '(タスク結果なし)')[:100]}...")
    print(f"  最終回答: {final_state_swarm.get('final_answer', '(最終回答なし)')}")
    assert final_state_swarm.get('final_answer') is not None or final_state_swarm.get('assigned_worker') == 'FINISH'

print("\nアサーション成功（各クエリで何らかの最終回答またはFINISH状態を確認）。")
``````
</details>

## 問題003: スーパーバイザーによる逐次連携ワークフロー

### 課題
問題002のスーパーバイザーを拡張し、複数のワーカーエージェントが順番に処理を行う逐次連携ワークフローを指示・管理できるようにします。例えば、「まずリサーチャーが情報を集め、次にその結果を元にライターが記事を書き、最後に校正者が校正する」といった流れです。スーパーバイザーは、現在のタスクが完了したら次にどのワーカーを呼び出すべきかを判断し、状態を更新して処理を繋げます。

*   **学習内容:** スーパーバイザーが複数ステップの計画（どのワーカーをどの順番で呼び出すか）を持ち、状態（例: 現在のステップ、全ステップリストなど）に基づいて逐次的にワーカーを呼び出す方法を学びます。これにより、より複雑なタスクの自動化が可能になります。

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated, List, Optional, Literal
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

SequentialWorker = Literal["Planner", "Researcher", "Writer", "Corrector", "Supervisor", "FINISH_SEQ"]

# --- 状態定義 (State) ---
class SequentialWorkflowState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    user_main_goal: str
    plan: Optional[List[SequentialWorker]] # 実行するワーカーの順序リスト
    current_worker_index: int
    current_task_data: Optional[str] # 現在のワーカーへの入力データ
    intermediate_results: dict # 各ワーカーの結果を保存
    final_product: Optional[str]

# --- ワーカーノード (ダミー) ---
def planner_node(state: SequentialWorkflowState):
    goal = state["user_main_goal"]
    print(f"Planner: 「{goal}」の計画を立案中...")
    # ダミープラン: Researcher -> Writer -> Corrector
    plan_steps: List[SequentialWorker] = ["Researcher", "Writer", "Corrector"]
    return {"plan": plan_steps, "current_worker_index": 0, "intermediate_results": {}, "messages":[AIMessage(content=f"計画: {plan_steps}", name="Planner")]}

def researcher_seq_node(state: SequentialWorkflowState):
    task = state["current_task_data"]
    print(f"Researcher (Sequential): 「{task}」の調査中...")
    result = f"「{task}」に関する詳細な調査データです。"
    new_results = state["intermediate_results"].copy()
    new_results["research_data"] = result
    return {"intermediate_results": new_results, "messages":[AIMessage(content=result, name="Researcher")]}

def writer_seq_node(state: SequentialWorkflowState):
    research_data = state["intermediate_results"].get("research_data", "調査データなし")
    print(f"Writer (Sequential): 「{research_data[:30]}...」を元に執筆中...")
    article = f"記事本文: {research_data} に基づく洞察。"
    new_results = state["intermediate_results"].copy()
    new_results["draft_article"] = article
    return {"intermediate_results": new_results, "messages":[AIMessage(content=article, name="Writer")]}

def corrector_seq_node(state: SequentialWorkflowState):
    draft = state["intermediate_results"].get("draft_article", "記事ドラフトなし")
    print(f"Corrector (Sequential): 「{draft[:30]}...」を校正中...")
    corrected = draft + " (校正済み)"
    return {"final_product": corrected, "messages":[AIMessage(content=corrected, name="Corrector")]}

# --- スーパーバイザー (逐次連携版) ---
def sequential_supervisor_node(state: SequentialWorkflowState):
    print("\nSequential Supervisor: 次のステップを決定...")
    plan = state.get("plan")
    if not plan: # 初回呼び出し (プランナーを起動)
        print("  -> プラン未作成。Plannerを起動します。")
        return {"current_task_data": state["user_main_goal"], "messages":[AIMessage(content="プランナーに計画作成を指示", name="Supervisor")]}

    idx = state.get("current_worker_index", 0)
    if idx < len(plan):
        next_worker_in_plan = plan[idx]
        # 次のワーカーへの入力データを準備 (ここでは前の結果をそのまま渡すなど、適宜調整)
        # 簡単のため、ここでは固定のタスク記述か、直前の結果を使う想定
        task_input_for_next = state["user_main_goal"] # デフォルト
        if next_worker_in_plan == "Writer" and "research_data" in state["intermediate_results"]:
            task_input_for_next = state["intermediate_results"]["research_data"]
        elif next_worker_in_plan == "Corrector" and "draft_article" in state["intermediate_results"]:
            task_input_for_next = state["intermediate_results"]["draft_article"]
        
        print(f"  -> 次のワーカー: {next_worker_in_plan}, 入力データ(一部): {str(task_input_for_next)[:30]}...")
        return {"current_task_data": task_input_for_next, "current_worker_index": idx + 1, "messages":[AIMessage(content=f"{next_worker_in_plan}に処理を指示", name="Supervisor")]}
    else:
        print("  -> 全ての計画ステップ完了。終了します。")
        return {"messages":[AIMessage(content="全ステップ完了。最終成果物を確認してください。", name="Supervisor")]}

# --- ルーター関数 (スーパーバイザーの指示に従う) ---
def sequential_router(state: SequentialWorkflowState):
    if not state.get("plan"):
        return "to_planner" # プランがなければプランナーへ
    
    idx = state.get("current_worker_index", 0) -1 # supervisorでインクリメント済みなので-1
    if idx < len(state["plan"]):
        worker_to_call = state["plan"][idx]
        print(f"Sequential Router: 次は {worker_to_call} を呼び出します。")
        if worker_to_call == "Researcher": return "to_researcher_seq"
        if worker_to_call == "Writer": return "to_writer_seq"
        if worker_to_call == "Corrector": return "to_corrector_seq"
    print("Sequential Router: 完了または不明なため終了します。")
    return END

# --- グラフ構築 ---
workflow = StateGraph(SequentialWorkflowState)
workflow.add_node("supervisor_seq", sequential_supervisor_node)
workflow.add_node("planner_seq", planner_node)
workflow.add_node("researcher_seq", researcher_seq_node)
workflow.add_node("writer_seq", writer_seq_node)
workflow.add_node("corrector_seq", corrector_seq_node)

workflow.set_entry_point("supervisor_seq")
workflow.add_conditional_edges(
    "supervisor_seq", sequential_router,
    {"to_planner": "planner_seq", "to_researcher_seq": "researcher_seq", 
     "to_writer_seq": "writer_seq", "to_corrector_seq": "corrector_seq", END: END}
)
# 各ノードは処理後Supervisorに戻る
workflow.add_edge("planner_seq", "supervisor_seq")
workflow.add_edge("researcher_seq", "supervisor_seq")
workflow.add_edge("writer_seq", "supervisor_seq")
workflow.add_edge("corrector_seq", "supervisor_seq")

graph = workflow.compile()

# --- 実行 --- 
goal = "新しいAI技術に関するブログ記事を作成する"
inputs_seq = {"messages": [HumanMessage(content=goal)], "user_main_goal": goal}
for event in graph.stream(inputs_seq, {"recursion_limit": 10}): print(event)
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated, List, Optional, Literal
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import Image, display

SequentialWorkerType = Literal["Planner", "Researcher", "Writer", "Editor", "Supervisor", "FINISH_WORKFLOW"]

# --- 状態定義 (State) ---
class SequentialWorkflowState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    user_goal: str                           # ユーザーの最終的な目標
    overall_plan: Optional[List[SequentialWorkerType]] # 実行するワーカーの順序リスト
    current_step_index: int                  # 現在の計画ステップのインデックス
    current_task_input: Optional[str]        # 現在のワーカーへの入力データ
    intermediate_data: dict                  # 各ワーカーの成果物を格納する辞書
    final_product: Optional[str]             # 最終成果物
    supervisor_next_action: Optional[SequentialWorkerType] # スーパーバイザーが次に指示するワーカー

# --- ワーカーノードのダミー実装 ---
def planner_node_seq(state: SequentialWorkflowState):
    goal = state["user_goal"]
    print(f"\nPlanner Node: 目標「{goal}」のための実行計画を立案します...")
    # ダミープラン: この順序でワーカーを実行する
    # 実際にはLLMでユーザーゴールに基づいて動的に計画を生成する
    plan_steps: List[SequentialWorkerType] = ["Researcher", "Writer", "Editor"]
    print(f"  -> 生成された計画: {plan_steps}")
    return {
        "overall_plan": plan_steps,
        "current_step_index": 0, # 計画の最初のステップから開始
        "intermediate_data": {},
        "messages": [AIMessage(content=f"立案された計画: {', '.join(plan_steps)}", name="Planner")]
    }

def researcher_node_seq(state: SequentialWorkflowState):
    task_input = state.get("current_task_input", state["user_goal"])
    print(f"\nResearcher Node (Sequential): タスク「{task_input}」について調査します...")
    research_output = f"「{task_input}」に関する詳細な調査データ。主要な発見はX, Y, Zです。"
    print(f"  -> 調査結果: {research_output}")
    updated_data = state.get("intermediate_data", {}).copy()
    updated_data["research_findings"] = research_output
    return {"intermediate_data": updated_data, "messages": [AIMessage(content=research_output, name="Researcher")]}

def writer_node_seq(state: SequentialWorkflowState):
    task_input = state.get("intermediate_data", {}).get("research_findings", "(調査データなし)")
    print(f"\nWriter Node (Sequential): 入力「{task_input[:50]}...」に基づいて記事を執筆します...")
    draft_article_output = f"# 分析記事\n\n## 序論\n{task_input[:70]}...\n\n## 本論\n...（詳細な議論）...\n\n##結論\n以上より、重要な示唆が得られました。"
    print(f"  -> 執筆された記事ドラフト: {draft_article_output[:100]}...")
    updated_data = state.get("intermediate_data", {}).copy()
    updated_data["article_draft"] = draft_article_output
    return {"intermediate_data": updated_data, "messages": [AIMessage(content=draft_article_output, name="Writer")]}

def editor_node_seq(state: SequentialWorkflowState):
    task_input = state.get("intermediate_data", {}).get("article_draft", "(記事ドラフトなし)")
    print(f"\nEditor Node (Sequential): ドラフト「{task_input[:50]}...」を校正・編集します...")
    edited_article_output = task_input.replace("重要な示唆", "極めて重要な示唆") + "\n\n(編集済み - Editor)"
    print(f"  -> 編集済み記事: {edited_article_output[:100]}...")
    # 最終成果物として final_product に格納
    return {"final_product": edited_article_output, "messages": [AIMessage(content=edited_article_output, name="Editor")]}

# --- スーパーバイザーノード (逐次連携版) ---
def sequential_supervisor_node(state: SequentialWorkflowState):
    print(f"\nSequential Supervisor: ワークフローの状態を確認し、次のアクションを指示します...")
    plan = state.get("overall_plan")
    current_idx = state.get("current_step_index", 0)

    if not plan: # 初回呼び出し時 (プランナーを起動)
        print("  -> 計画がまだありません。Plannerを起動します。")
        return {"supervisor_next_action": "Planner", "current_task_input": state["user_goal"]}

    if current_idx < len(plan):
        next_worker_in_plan = plan[current_idx]
        print(f"  -> 計画のステップ {current_idx + 1}/{len(plan)}: 次のワーカーは {next_worker_in_plan}")
        
        # 次のワーカーへの入力データを準備
        task_data_for_next_worker = state["user_goal"] # デフォルトは元のゴール
        if next_worker_in_plan == "Writer":
            task_data_for_next_worker = state.get("intermediate_data", {}).get("research_findings", state["user_goal"])
        elif next_worker_in_plan == "Editor":
            task_data_for_next_worker = state.get("intermediate_data", {}).get("article_draft", state["user_goal"])
        
        print(f"    タスク入力(一部): {str(task_data_for_next_worker)[:50]}...")
        return {"supervisor_next_action": next_worker_in_plan, "current_task_input": task_data_for_next_worker, "current_step_index": current_idx + 1}
    else:
        print("  -> 全ての計画ステップが完了しました。ワークフローを終了します。")
        final_prod = state.get("final_product", "(最終成果物なし)")
        return {"supervisor_next_action": "FINISH_WORKFLOW", "messages": [AIMessage(content=f"全ステップ完了。最終成果物: {final_prod[:100]}...", name="Supervisor")]}

# --- ルーター関数 (スーパーバイザーの指示に基づいてルーティング) ---
def supervisor_sequential_router(state: SequentialWorkflowState):
    next_action = state.get("supervisor_next_action")
    print(f"\nSupervisor Sequential Router: 次のアクション -> {next_action}")
    if next_action == "Planner": return "route_to_planner"
    if next_action == "Researcher": return "route_to_researcher"
    if next_action == "Writer": return "route_to_writer"
    if next_action == "Editor": return "route_to_editor"
    if next_action == "FINISH_WORKFLOW": return END
    print("  -> 不明なアクションまたは終了指示。ENDへ")
    return END

# --- グラフ構築 ---
workflow = StateGraph(SequentialWorkflowState)

workflow.add_node("supervisor_sequential", sequential_supervisor_node)
workflow.add_node("planner_sequential", planner_node_seq)
workflow.add_node("researcher_sequential", researcher_node_seq)
workflow.add_node("writer_sequential", writer_node_seq)
workflow.add_node("editor_sequential", editor_node_seq)

workflow.set_entry_point("supervisor_sequential") # スーパーバイザーから開始

workflow.add_conditional_edges(
    "supervisor_sequential", 
    supervisor_sequential_router,
    {
        "route_to_planner": "planner_sequential",
        "route_to_researcher": "researcher_sequential", 
        "route_to_writer": "writer_sequential", 
        "route_to_editor": "editor_sequential", 
        END: END
    }
)

# 各ワーカーノードは処理後、スーパーバイザーに戻る
workflow.add_edge("planner_sequential", "supervisor_sequential")
workflow.add_edge("researcher_sequential", "supervisor_sequential")
workflow.add_edge("writer_sequential", "supervisor_sequential")
workflow.add_edge("editor_sequential", "supervisor_sequential")

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
user_main_goal_request = "AIが人間社会に与える影響についての包括的なレポートを作成する"
print(f"\n--- スーパーバイザーによる逐次連携ワークフローテスト (目標: '{user_main_goal_request}') ---")

inputs_sequential_flow = {"messages": [HumanMessage(content=user_main_goal_request)], "user_goal": user_main_goal_request}

print("\nストリーム出力:")
for i, event in enumerate(graph.stream(inputs_sequential_flow, {"recursion_limit": 15})): # ステップ数が多いのでlimit調整
    print(f"Event {i+1}: {event}")

final_state_seq_flow = graph.invoke(inputs_sequential_flow, {"recursion_limit": 15})
print("\n最終状態の確認:")
print(f"  ユーザー目標: {final_state_seq_flow['user_goal']}")
print(f"  実行された計画: {final_state_seq_flow.get('overall_plan')}")
print(f"  最終成果物: {final_state_seq_flow.get('final_product')}")
print(f"  スーパーバイザーの最後の指示: {final_state_seq_flow.get('supervisor_next_action')}")

assert final_state_seq_flow.get('final_product') is not None
assert final_state_seq_flow.get('supervisor_next_action') == "FINISH_WORKFLOW"
assert "(編集済み - Editor)" in final_state_seq_flow.get('final_product', "")
print("\nアサーション成功！")
``````
</details>

## 問題004: 階層型エージェント - タスクの委任と報告

### 課題
より複雑な問題解決のために、エージェントが階層構造をとり、上位のエージェント（マネージャー）が下位のエージェント（ワーカー）にタスクを委任し、ワーカーはその結果をマネージャーに報告するという形が考えられます。この問題では、マネージャーエージェントとワーカーエージェント（例: 特定の専門知識を持つリサーチャー）を定義し、マネージャーがタスクを分解してワーカーに割り振り、ワーカーの報告をまとめて最終結果とするフローを構築します。

*   **学習内容:** マネージャー役のLLMノードがタスクをサブタスクに分解し、それを別のワーカー役LLMノードに（状態を通じて）渡し、ワーカーの実行結果をマネージャーが再度集約するという、階層的な処理の流れを実装します。状態設計がここでも重要になります。

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated, List, Optional, Dict
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# --- 状態定義 (State) ---
class HierarchicalAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    main_task: str
    sub_tasks: Optional[List[Dict[str, str]]] # 例: [{'id':'task1', 'description':'...', 'assigned_to':None, 'result':None}]
    current_sub_task_id: Optional[str]
    final_report: Optional[str]
    manager_action: Optional[str] # 'delegate', 'summarize', 'finish'

# --- ノード定義 ---
def manager_agent_node(state: HierarchicalAgentState):
    print("\nManager Agent: タスクを評価・指示します...")
    if not state.get("sub_tasks"): # 初回: タスク分解と委任先決定
        print(f"  -> メインタスク「{state['main_task']}」をサブタスクに分解中...")
        # ダミーサブタスク (実際にはLLMで分解)
        sub_tasks_list = [
            {"id": "sub1", "description": f"{state['main_task']}の背景調査", "assigned_to": "Worker", "result": None},
            {"id": "sub2", "description": f"{state['main_task']}の主要要素分析", "assigned_to": "Worker", "result": None}
        ]
        next_sub_task_id = sub_tasks_list[0]['id'] if sub_tasks_list else None
        return {"sub_tasks": sub_tasks_list, "current_sub_task_id": next_sub_task_id, "manager_action": "delegate"}
    else: # サブタスクの結果を評価し、次を決定
        all_done = all(st.get("result") for st in state["sub_tasks"])
        if all_done:
            print("  -> 全サブタスク完了。最終報告をまとめます。")
            # ダミー報告 (実際にはLLMで集約)
            report = f"メインタスク「{state['main_task']}」の報告:\n"
            for st in state["sub_tasks"]:
                report += f"  - {st['description']} 結果: {st['result']}\n"
            return {"final_report": report, "manager_action": "finish"}
        else: # まだ実行すべきサブタスクがある
            next_task = next((st for st in state["sub_tasks"] if not st.get("result")), None)
            if next_task:
                 print(f"  -> 次のサブタスク「{next_task['description']}」を委任します。")
                 return {"current_sub_task_id": next_task['id'], "manager_action": "delegate"}
    return {"manager_action": "finish"} # フォールバック

def worker_agent_node(state: HierarchicalAgentState):
    current_id = state["current_sub_task_id"]
    sub_task_detail = next(st for st in state["sub_tasks"] if st['id'] == current_id)
    print(f"\nWorker Agent: サブタスク「{sub_task_detail['description']}」を実行中...")
    # ダミー実行結果
    result = f"「{sub_task_detail['description']}」の実行結果です。"
    updated_sub_tasks = []
    for st in state["sub_tasks"]:
        if st['id'] == current_id:
            updated_sub_tasks.append({**st, "result": result})
        else:
            updated_sub_tasks.append(st)
    return {"sub_tasks": updated_sub_tasks, "messages":[AIMessage(content=result, name="Worker")]}

# --- ルーター関数 ---
def manager_router(state: HierarchicalAgentState):
    action = state.get("manager_action")
    print(f"Manager Router: アクション '{action}' に基づきルーティング")
    if action == "delegate" and state.get("current_sub_task_id"):
        return "to_worker"
    if action == "finish":
        return END
    return END # デフォルト

# --- グラフ構築 ---
workflow = StateGraph(HierarchicalAgentState)
workflow.add_node("manager", manager_agent_node)
workflow.add_node("worker", worker_agent_node)

workflow.set_entry_point("manager")
workflow.add_conditional_edges("manager", manager_router, {"to_worker": "worker", END: END})
workflow.add_edge("worker", "manager") # Workerは結果をManagerに報告

graph = workflow.compile()

# --- 実行 ---
main_task_desc = "競合他社の新しいマーケティング戦略の分析"
inputs_hierarchical = {"messages":[HumanMessage(content=main_task_desc)], "main_task": main_task_desc}
for event in graph.stream(inputs_hierarchical, {"recursion_limit": 10}): print(event)
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated, List, Optional, Dict
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import Image, display
import uuid # サブタスクID用

# --- サブタスクの状態を定義 ---
class SubTask(TypedDict):
    id: str
    description: str
    assigned_to: Literal["MarketAnalystWorker", "TechnicalWriterWorker"] # ワーカーの種類
    status: Literal["pending", "in_progress", "completed", "failed"]
    result: Optional[str]

# --- 全体の状態定義 (State) ---
class HierarchicalAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    main_user_request: str               # ユーザーからの主要なリクエスト
    sub_tasks_list: Optional[List[SubTask]] # マネージャーが生成するサブタスクのリスト
    current_processing_task_id: Optional[str] # 現在処理中のサブタスクID
    final_compiled_report: Optional[str]   # 全サブタスク完了後の最終報告書
    manager_next_step: Optional[Literal["delegate_task", "compile_report", "finish_workflow"]] # マネージャーの次の行動

# --- ワーカーエージェントのダミー実装ノード ---
def market_analyst_worker_node(state: HierarchicalAgentState):
    task_id = state["current_processing_task_id"]
    task_desc = "不明な分析タスク"
    if state.get("sub_tasks_list") and task_id:
        current_task_obj = next((st for st in state["sub_tasks_list"] if st["id"] == task_id), None)
        if current_task_obj: task_desc = current_task_obj["description"]
    
    print(f"\nMarket Analyst Worker: サブタスク「{task_desc}」(ID: {task_id}) を実行します...")
    # ダミーの市場分析結果
    analysis_result = f"市場分析結果 for '{task_desc}': 主要トレンドはX、競合の動きはY、推奨アクションはZ。"
    print(f"  -> 分析結果: {analysis_result}")
    
    # 対応するサブタスクの結果を更新
    updated_tasks = []
    if state.get("sub_tasks_list"):
        for st in state["sub_tasks_list"]:
            if st["id"] == task_id:
                updated_tasks.append({**st, "status": "completed", "result": analysis_result})
            else:
                updated_tasks.append(st)
                
    return {"sub_tasks_list": updated_tasks, "messages": [AIMessage(content=analysis_result, name="MarketAnalystWorker")]}

def technical_writer_worker_node(state: HierarchicalAgentState):
    task_id = state["current_processing_task_id"]
    task_input_data = "不明な技術文書作成タスク"
    if state.get("sub_tasks_list") and task_id:
        current_task_obj = next((st for st in state["sub_tasks_list"] if st["id"] == task_id), None)
        if current_task_obj: task_input_data = current_task_obj["description"]
        # 実際には、他のサブタスクの結果 (例: MarketAnalystの結果) を current_task_input として渡す
        # ここでは簡略化のため、descriptionをそのまま使う

    print(f"\nTechnical Writer Worker: サブタスク「{task_input_data}」(ID: {task_id}) のための技術文書を作成します...")
    document_content = f"## 技術文書: {task_input_data}\n\nこれは詳細な技術仕様と実装ガイドです。...（文書本文）..."
    print(f"  -> 作成された文書: {document_content[:80]}...")
    
    updated_tasks = []
    if state.get("sub_tasks_list"):
        for st in state["sub_tasks_list"]:
            if st["id"] == task_id:
                updated_tasks.append({**st, "status": "completed", "result": document_content})
            else:
                updated_tasks.append(st)
    return {"sub_tasks_list": updated_tasks, "messages": [AIMessage(content=document_content, name="TechnicalWriterWorker")]}

# --- マネージャーエージェントノード ---
def manager_agent_node(state: HierarchicalAgentState):
    print(f"\nManager Agent: ワークフローの状態を評価し、次の指示を出します...")
    sub_tasks = state.get("sub_tasks_list")
    main_req = state["main_user_request"]

    if not sub_tasks: # 初回呼び出し時: サブタスクを計画・生成
        print(f"  -> メインリクエスト「{main_req}」に対するサブタスクを計画します。")
        # LLMでサブタスクリストを生成 (ここではダミー)
        # plan_response = llm.invoke(f"「{main_req}」を達成するためのサブタスクと担当ワーカーを提案してください。例: ['タスク説明1 (MarketAnalystWorker)', 'タスク説明2 (TechnicalWriterWorker)']")
        # parsed_sub_tasks = parse_plan_response(plan_response.content) # 実際にはパース処理が必要
        generated_sub_tasks: List[SubTask] = [
            {"id": str(uuid.uuid4()), "description": f"{main_req} の市場調査と競合分析", "assigned_to": "MarketAnalystWorker", "status": "pending", "result": None},
            {"id": str(uuid.uuid4()), "description": f"{main_req} の技術仕様書の作成", "assigned_to": "TechnicalWriterWorker", "status": "pending", "result": None}
        ]
        print(f"  -> 生成されたサブタスク: {generated_sub_tasks}")
        return {"sub_tasks_list": generated_sub_tasks, "manager_next_step": "delegate_task"}

    # 実行すべき次のサブタスクを探す
    next_pending_task: Optional[SubTask] = None
    for task in sub_tasks:
        if task["status"] == "pending":
            next_pending_task = task
            break
            
    if next_pending_task:
        print(f"  -> 次のサブタスク「{next_pending_task['description']}」(ID: {next_pending_task['id']}) をワーカー「{next_pending_task['assigned_to']}」に委任します。")
        # サブタスクのステータスを更新 (オプション)
        # updated_tasks = [{**t, "status": "in_progress"} if t["id"] == next_pending_task["id"] else t for t in sub_tasks]
        return {"current_processing_task_id": next_pending_task["id"], "manager_next_step": "delegate_task"}
    else: # 全てのサブタスクがpending以外 (完了または失敗)
        print("  -> 全てのサブタスクが処理されました。最終報告書を作成します。")
        # LLMで全サブタスクの結果を統合して最終報告書を作成
        report_parts = []
        for task in sub_tasks:
            report_parts.append(f"サブタスク「{task['description']}」({task['assigned_to']}):\n{task.get('result', '(結果なし)')}")
        final_report_content = f"## メインリクエスト「{main_req}」に関する最終報告\n\n" + "\n\n---\n\n".join(report_parts)
        print(f"  -> 生成された最終報告書: {final_report_content[:150]}...")
        return {"final_compiled_report": final_report_content, "manager_next_step": "finish_workflow"}

# --- ルーター関数 (マネージャーの指示に基づいてルーティング) ---
def manager_hierarchical_router(state: HierarchicalAgentState):
    next_step_action = state.get("manager_next_step")
    current_task_id_to_process = state.get("current_processing_task_id")
    print(f"\nManager Hierarchical Router: 次のステップアクション -> {next_step_action}")
    
    if next_step_action == "delegate_task" and current_task_id_to_process:
        # 委任するワーカーを特定
        task_to_delegate = next((st for st in state.get("sub_tasks_list", []) if st["id"] == current_task_id_to_process), None)
        if task_to_delegate:
            worker_name = task_to_delegate["assigned_to"]
            print(f"  -> ワーカー「{worker_name}」にルーティングします (タスクID: {current_task_id_to_process})")
            if worker_name == "MarketAnalystWorker": return "route_to_market_analyst"
            if worker_name == "TechnicalWriterWorker": return "route_to_technical_writer"
    
    if next_step_action == "finish_workflow" or next_step_action == "compile_report": # compile_reportは実質finishの前段階
        print("  -> ワークフロー終了へ")
        return END
        
    print("  -> 不明なアクションまたは継続不可。暫定的にマネージャーに処理を戻します（要デバッグ）。")
    # 本来はエラー処理か、より明確なフォールバックが必要
    return "loop_back_to_manager_for_re_evaluation" 

# --- グラフ構築 ---
workflow = StateGraph(HierarchicalAgentState)

workflow.add_node("manager_main", manager_agent_node)
workflow.add_node("market_analyst", market_analyst_worker_node)
workflow.add_node("tech_writer", technical_writer_worker_node)

workflow.set_entry_point("manager_main")

workflow.add_conditional_edges(
    "manager_main",
    manager_hierarchical_router,
    {
        "route_to_market_analyst": "market_analyst",
        "route_to_technical_writer": "tech_writer",
        "loop_back_to_manager_for_re_evaluation": "manager_main", # エラーや予期せぬ状況のフォールバック
        END: END
    }
)

# 各ワーカーは処理後、マネージャーに報告（結果を返す）
workflow.add_edge("market_analyst", "manager_main")
workflow.add_edge("tech_writer", "manager_main")

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
main_task_for_hierarchy = "新しいスマートフォンアプリの市場投入戦略を立案し、関連技術文書を作成する"
print(f"\n--- 階層型エージェントテスト (メインタスク: '{main_task_for_hierarchy}') ---")

inputs_hierarchical_flow = {"messages": [HumanMessage(content=main_task_for_hierarchy)], "main_user_request": main_task_for_hierarchy}

print("\nストリーム出力:")
for i, event in enumerate(graph.stream(inputs_hierarchical_flow, {"recursion_limit": 10})): # サブタスク数に応じて調整
    print(f"Event {i+1}: {event}")

final_state_hierarchy = graph.invoke(inputs_hierarchical_flow, {"recursion_limit": 10})
print("\n最終状態の確認:")
print(f"  メインリクエスト: {final_state_hierarchy['main_user_request']}")
print(f"  サブタスクリスト ({len(final_state_hierarchy.get('sub_tasks_list', []))}件):")
for st_idx, st_item in enumerate(final_state_hierarchy.get('sub_tasks_list', [])):
    print(f"    {st_idx+1}. ID: {st_item['id']}, Desc: {st_item['description'][:30]}..., Assignee: {st_item['assigned_to']}, Status: {st_item['status']}, Result(part): {str(st_item.get('result'))[:40]}...")
print(f"  最終報告書(一部): {final_state_hierarchy.get('final_compiled_report', '(報告書なし)')[:200]}...")
print(f"  マネージャーの最終アクション: {final_state_hierarchy.get('manager_next_step')}")

assert final_state_hierarchy.get('final_compiled_report') is not None
assert final_state_hierarchy.get('manager_next_step') == "finish_workflow"
assert all(task.get("status") == "completed" for task in final_state_hierarchy.get("sub_tasks_list", [])), "全てのサブタスクが完了していません。"
print("\nアサーション成功！")
``````
</details>

## 問題005: 第4章のまとめ - 複数エージェントによる協調型リサーチボット

### 課題
第4章で学んだマルチエージェントの概念（役割分担、スーパーバイザーによるタスク割り振り、逐次連携、階層構造など）を組み合わせて、より高度な協調型リサーチボットを構築します。このボットは、ユーザーからの複雑な調査リクエストに対し、以下のようなステップで応答を生成することを目指します。
1.  **リクエスト分析・計画フェーズ (プランナー/スーパーバイザー):** ユーザーのリクエストを理解し、必要な情報収集タスクを特定し、実行計画（どの専門リサーチャーに何を調査させるかなど）を立てます。
2.  **並列情報収集フェーズ (専門リサーチャー群):** 計画に基づき、複数の専門リサーチャー（例: Web検索リサーチャー、特定DBリサーチャーなど）が並列で情報を収集します。（この問題ではダミーの専門リサーチャーを1-2種類とします）
3.  **情報統合・報告書作成フェーズ (ライター/アナリスト):** 各リサーチャーからの情報を統合し、矛盾点を解消したり、洞察を加えたりして、最終的な報告書を作成します。
4.  **最終出力:** 生成された報告書をユーザーに提示します。

*   **学習内容:** これまでに学んだマルチエージェントのパターンを組み合わせ、状態管理を工夫することで、複数のエージェントが動的に連携し、一つの目標に向かって協調作業を行うワークフローを構築します。特に、スーパーバイザーが全体の進行を管理し、ワーカーエージェントが専門タスクを処理し、その結果を再びスーパーバイザー（または別の中間エージェント）が集約するという流れを体験します。

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated, List, Optional, Dict, Literal
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun # Web検索用
import uuid

# --- ツール準備 ---
web_search_tool = DuckDuckGoSearchRun()

# --- 状態定義 (State) ---
AgentName = Literal["PlannerSupervisor", "WebSearcher", "ReportWriter", "FINISH_COOP"]
SubTaskStatus = Literal["pending", "in_progress", "completed", "failed"]

class CoopSubTask(TypedDict):
    id: str
    description: str
    assigned_to: AgentName
    status: SubTaskStatus
    result: Optional[str]
    dependencies: Optional[List[str]] # このタスクが依存する他のサブタスクID

class CooperativeResearchBotState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    main_research_request: str
    research_plan: Optional[List[CoopSubTask]] # 実行すべきサブタスクのリスト
    active_sub_task_id: Optional[str] # 現在処理中のサブタスクID
    # intermediate_results: Dict[str, str] # 各サブタスクの結果 (sub_tasks_listのresultに集約)
    compiled_report: Optional[str]
    supervisor_next_action_coop: Optional[AgentName]

# --- エージェントノード (LLMやツールを使う) ---
def planner_supervisor_node(state: CooperativeResearchBotState):
    print("\nPlanner/Supervisor: 状況を評価し、計画または次の指示を出します...")
    user_req = state["main_research_request"]
    plan = state.get("research_plan")

    if not plan: # 初回: 計画立案
        print(f"  -> リクエスト「{user_req}」の調査計画を作成します。")
        # ダミープラン (実際にはLLMでリクエスト内容から動的に生成)
        task1_id = str(uuid.uuid4())
        task2_id = str(uuid.uuid4())
        new_plan: List[CoopSubTask] = [
            {"id": task1_id, "description": f"「{user_req}」に関する基本情報をWebで検索", "assigned_to": "WebSearcher", "status": "pending", "result": None, "dependencies": []},
            {"id": task2_id, "description": f"収集した情報を基に、{user_req}に関する包括的なレポートを作成", "assigned_to": "ReportWriter", "status": "pending", "result": None, "dependencies": [task1_id]}
        ]
        return {"research_plan": new_plan, "supervisor_next_action_coop": "PlannerSupervisor"} # 再度Supervisorで委任先決定

    # 実行可能な次のタスクを探す (依存関係も考慮)
    next_task_to_run: Optional[CoopSubTask] = None
    for task in plan:
        if task["status"] == "pending":
            deps_completed = True
            if task.get("dependencies"):
                for dep_id in task["dependencies"]:
                    dep_task = next((t for t in plan if t["id"] == dep_id), None)
                    if not dep_task or dep_task["status"] != "completed":
                        deps_completed = False
                        break
            if deps_completed:
                next_task_to_run = task
                break
    
    if next_task_to_run:
        print(f"  -> 次のサブタスク「{next_task_to_run['description']}」を「{next_task_to_run['assigned_to']}」に委任します。")
        return {"active_sub_task_id": next_task_to_run["id"], "supervisor_next_action_coop": next_task_to_run["assigned_to"]}
    else: # 全タスクがpending以外 (完了 or 失敗)
        all_completed = all(t["status"] == "completed" for t in plan)
        if all_completed:
            print("  -> 全サブタスク完了。最終成果物はReportWriterの成果物とします。")
            final_report_task = next((t for t in plan if t["assigned_to"] == "ReportWriter"), None)
            final_content = final_report_task['result'] if final_report_task else "レポート作成失敗"
            return {"compiled_report": final_content, "supervisor_next_action_coop": "FINISH_COOP"}
        else:
            print("  -> 実行可能なタスクなし、または失敗あり。ワークフロー終了。")
            return {"supervisor_next_action_coop": "FINISH_COOP", "compiled_report": "一部タスクが未完了または失敗しました。"}

def web_searcher_node(state: CooperativeResearchBotState):
    task_id = state["active_sub_task_id"]
    task_desc = "不明な検索タスク"
    if state.get("research_plan") and task_id:
        current_task_obj = next((st for st in state["research_plan"] if st["id"] == task_id), None)
        if current_task_obj: task_desc = current_task_obj["description"]
    print(f"\nWebSearcher: 「{task_desc}」のWeb検索を実行します...")
    search_result = web_search_tool.invoke(task_desc) # 実際の検索ツール呼び出し
    print(f"  -> 検索結果(一部): {search_result[:100]}...")
    updated_plan = []
    if state.get("research_plan"):
        for st in state["research_plan"]:
            if st["id"] == task_id:
                updated_plan.append({**st, "status": "completed", "result": search_result})
            else: updated_plan.append(st)
    return {"research_plan": updated_plan, "supervisor_next_action_coop": "PlannerSupervisor", "messages":[AIMessage(content=search_result, name="WebSearcher")]}

def report_writer_node(state: CooperativeResearchBotState):
    task_id = state["active_sub_task_id"]
    task_desc = "不明なレポート作成タスク"
    input_data_for_writer = ""
    if state.get("research_plan") and task_id:
        current_task_obj = next((st for st in state["research_plan"] if st["id"] == task_id), None)
        if current_task_obj:
            task_desc = current_task_obj["description"]
            # 依存関係のあるタスクの結果を取得
            if current_task_obj.get("dependencies"):
                for dep_id in current_task_obj["dependencies"]:
                    dep_task = next((t for t in state["research_plan"] if t["id"] == dep_id), None)
                    if dep_task and dep_task.get("result"):
                        input_data_for_writer += f"\n--- {dep_task['description']}の結果 ---\n{dep_task['result']}\n"
    if not input_data_for_writer: input_data_for_writer = "利用可能な事前データがありません。"
    
    print(f"\nReportWriter: 「{task_desc}」のレポートを作成します。入力データ(一部): {input_data_for_writer[:100]}...")
    # LLMでレポート作成
    # report_content = llm.invoke(f"以下の情報を元に「{task_desc}」というタイトルの報告書を作成してください。\n{input_data_for_writer}").content
    report_content = f"## レポート: {task_desc}\n\n{input_data_for_writer}\n\n以上が報告内容です。"
    print(f"  -> 作成されたレポート(一部): {report_content[:100]}...")
    updated_plan = []
    if state.get("research_plan"):
        for st in state["research_plan"]:
            if st["id"] == task_id:
                updated_plan.append({**st, "status": "completed", "result": report_content})
            else: updated_plan.append(st)
    return {"research_plan": updated_plan, "supervisor_next_action_coop": "PlannerSupervisor", "messages":[AIMessage(content=report_content, name="ReportWriter")]}

# --- ルーター ---
def coop_router(state: CooperativeResearchBotState):
    next_action = state.get("supervisor_next_action_coop")
    print(f"Coop Router: 次のアクション -> {next_action}")
    if next_action == "PlannerSupervisor": return "to_supervisor"
    if next_action == "WebSearcher": return "to_web_searcher"
    if next_action == "ReportWriter": return "to_report_writer"
    if next_action == "FINISH_COOP": return END
    return END # フォールバック

# --- グラフ構築 ---
workflow = StateGraph(CooperativeResearchBotState)
workflow.add_node("supervisor_coop", planner_supervisor_node)
workflow.add_node("web_searcher_coop", web_searcher_node)
workflow.add_node("report_writer_coop", report_writer_node)

workflow.set_entry_point("supervisor_coop")
workflow.add_conditional_edges("supervisor_coop", coop_router, 
                               {"to_supervisor":"supervisor_coop", "to_web_searcher":"web_searcher_coop", 
                                "to_report_writer":"report_writer_coop", END:END})
workflow.add_edge("web_searcher_coop", "supervisor_coop")
workflow.add_edge("report_writer_coop", "supervisor_coop")

graph = workflow.compile()

# --- 実行 ---
main_req = "LangGraphの最新のユースケースと将来性について調査し、レポートを作成してください。"
inputs_coop = {"messages":[HumanMessage(content=main_req)], "main_research_request": main_req}
for event in graph.stream(inputs_coop, {"recursion_limit":15}): print(event)
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated, List, Optional, Dict, Literal
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun # Web検索用
from IPython.display import Image, display
import uuid # サブタスクID用

# --- ツール準備 ---
web_search_tool = DuckDuckGoSearchRun()

# --- 状態定義 (State) ---
AgentName = Literal["PlannerSupervisor", "WebSearcher", "DataAnalyst", "ReportWriter", "FINISH_WORKFLOW"]
SubTaskStatus = Literal["pending", "in_progress", "completed", "failed"]

class CooperativeSubTask(TypedDict):
    id: str
    description: str                          # このサブタスクが何をするか
    assigned_to: AgentName                    # どのエージェントに割り当てるか
    status: SubTaskStatus                     # 現在のステータス
    result: Optional[str]                     # このサブタスクの成果物
    dependencies: Optional[List[str]]         # このタスクが依存する他のサブタスクIDのリスト
    input_data_source_ids: Optional[List[str]] # このタスクが入力として使う他のサブタスクIDのリスト

class CooperativeResearchBotState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    main_research_request: str                   # ユーザーからの元のリクエスト
    research_plan: Optional[List[CooperativeSubTask]] # 実行すべきサブタスクの計画リスト
    current_processing_sub_task_id: Optional[str] # 現在処理中のサブタスクID
    final_report_content: Optional[str]            # 最終的なレポート内容
    supervisor_next_directive: Optional[AgentName] # スーパーバイザーが次に起動するエージェント（または終了指示）

# --- エージェントノード (LLMやツールを使う) ---

def planner_supervisor_node(state: CooperativeResearchBotState):
    print("\nPlanner/Supervisor Agent: ワークフローの計画と進行管理を行います...")
    user_main_req = state["main_research_request"]
    current_plan = state.get("research_plan")

    if not current_plan: # 初回呼び出し時: 計画を立案
        print(f"  -> 初期リクエスト「{user_main_req}」に対する調査計画を立案します。")
        # ここでLLMを使ってリクエストを分析し、サブタスクのリストと依存関係を生成する
        # 例: plan_str = llm.invoke(f"「{user_main_req}」を達成するためのサブタスクリストと依存関係、担当エージェントを提案してください。利用可能なエージェント: WebSearcher, DataAnalyst, ReportWriter。JSON形式で出力してください。")
        # new_plan_from_llm = json.loads(plan_str.content) # 実際にはパースとCooperativeSubTaskへの変換が必要
        
        # ダミープラン
        task_search_id = str(uuid.uuid4())
        task_analyze_id = str(uuid.uuid4())
        task_write_id = str(uuid.uuid4())
        generated_plan: List[CooperativeSubTask] = [
            {"id": task_search_id, "description": f"「{user_main_req}」に関する基礎情報をWebで幅広く検索する", "assigned_to": "WebSearcher", "status": "pending", "result": None, "dependencies": [], "input_data_source_ids": []},
            {"id": task_analyze_id, "description": f"収集したWeb検索結果を分析し、主要なポイントと洞察を抽出する", "assigned_to": "DataAnalyst", "status": "pending", "result": None, "dependencies": [task_search_id], "input_data_source_ids": [task_search_id]},
            {"id": task_write_id, "description": f"分析結果と洞察に基づいて、{user_main_req}に関する包括的な報告書を作成する", "assigned_to": "ReportWriter", "status": "pending", "result": None, "dependencies": [task_analyze_id], "input_data_source_ids": [task_analyze_id]}
        ]
        print(f"  -> 生成された計画: {generated_plan}")
        # 最初のタスクを指示するために再度自身を呼び出すように設定
        return {"research_plan": generated_plan, "supervisor_next_directive": "PlannerSupervisor", "messages": [AIMessage(content=f"計画を立案しました: {len(generated_plan)}ステップ", name="PlannerSupervisor")]}

    # 実行可能な次のタスクを探す (依存関係を考慮)
    next_task_to_delegate: Optional[CooperativeSubTask] = None
    for task_item in current_plan:
        if task_item["status"] == "pending":
            dependencies_met = True
            if task_item.get("dependencies"):
                for dep_id_val in task_item["dependencies"]:
                    dependent_task = next((t for t in current_plan if t["id"] == dep_id_val), None)
                    if not dependent_task or dependent_task["status"] != "completed":
                        dependencies_met = False
                        break
            if dependencies_met:
                next_task_to_delegate = task_item
                break
    
    if next_task_to_delegate:
        print(f"  -> 次のサブタスク「{next_task_to_delegate['description']}」をエージェント「{next_task_to_delegate['assigned_to']}」に委任します。")
        # タスクのステータスをin_progressに更新（オプション）
        # updated_plan_for_status = [{**t, "status": "in_progress"} if t["id"] == next_task_to_delegate["id"] else t for t in current_plan]
        return {"current_processing_sub_task_id": next_task_to_delegate["id"], "supervisor_next_directive": next_task_to_delegate["assigned_to"]}
    else: # 実行可能なペンディングタスクなし
        all_tasks_completed = all(t["status"] == "completed" for t in current_plan)
        if all_tasks_completed:
            print("  -> 全てのサブタスクが完了しました。最終報告書を確認し、ワークフローを終了します。")
            # 最終報告書は通常、ReportWriterの成果物
            final_report_task_obj = next((t for t in current_plan if t["assigned_to"] == "ReportWriter" and t["status"] == "completed"), None)
            final_report_str = final_report_task_obj['result'] if final_report_task_obj and final_report_task_obj['result'] else "最終報告書の生成に失敗しました。"
            return {"final_report_content": final_report_str, "supervisor_next_directive": "FINISH_WORKFLOW", "messages": [AIMessage(content=f"全タスク完了。最終報告: {final_report_str[:100]}...", name="PlannerSupervisor")]}
        else:
            # 失敗したタスクやデッドロックの可能性あり
            print("  -> 注意: 実行可能なペンディングタスクがありませんが、全タスク完了でもありません。問題が発生した可能性があります。")
            return {"supervisor_next_directive": "FINISH_WORKFLOW", "final_report_content": "エラー: ワークフローが途中で停止しました。"}

def web_searcher_node_coop(state: CooperativeResearchBotState):
    task_id = state["current_processing_sub_task_id"]
    task_object = next((st for st in state["research_plan"] if st["id"] == task_id), None)
    if not task_object: return {"supervisor_next_directive": "PlannerSupervisor"} # エラーケース
    task_description = task_object["description"]
    print(f"\nWebSearcher Agent: サブタスク「{task_description}」のWeb検索を実行します...")
    search_query = task_description # 簡単のため、タスク記述をそのままクエリに
    # search_result_str = web_search_tool.invoke(search_query)
    search_result_str = f"「{search_query}」に関するWeb検索結果: LangGraphはLLMアプリ構築ライブラリで、エージェントや複雑なフローを状態グラフで定義できます。最新版はvX.Y.Zで多くの機能改善が...（ダミー検索結果）。"
    print(f"  -> 検索結果(一部): {search_result_str[:100]}...")
    
    updated_plan_list = []
    for st_item in state["research_plan"]:
        if st_item["id"] == task_id:
            updated_plan_list.append({**st_item, "status": "completed", "result": search_result_str})
        else: updated_plan_list.append(st_item)
    return {"research_plan": updated_plan_list, "supervisor_next_directive": "PlannerSupervisor", "messages":[AIMessage(content=search_result_str, name="WebSearcher")]}

def data_analyst_node_coop(state: CooperativeResearchBotState):
    task_id = state["current_processing_sub_task_id"]
    task_object = next((st for st in state["research_plan"] if st["id"] == task_id), None)
    if not task_object: return {"supervisor_next_directive": "PlannerSupervisor"}
    task_description = task_object["description"]
    
    input_data_str = ""
    if task_object.get("input_data_source_ids"):
        for source_id in task_object["input_data_source_ids"]:
            source_task = next((t for t in state["research_plan"] if t["id"] == source_id), None)
            if source_task and source_task.get("result"):
                input_data_str += f"\n--- {source_task['description']} の結果 ---\n{source_task['result']}\n"
    if not input_data_str: input_data_str = "(分析対象データなし)"
    
    print(f"\nDataAnalyst Agent: サブタスク「{task_description}」のデータ分析を実行します。入力(一部): {input_data_str[:100]}...")
    # analysis_output = llm.invoke(f"以下のデータを分析し、主要な洞察を抽出してください: {input_data_str}").content
    analysis_output = f"データ分析結果 for '{task_description}': 主要な洞察として、トレンドAが確認され、要素BとCの間に強い相関が見られました。推奨アクションはDです（ダミー分析結果）。"
    print(f"  -> 分析結果: {analysis_output}")
    updated_plan_list = []
    for st_item in state["research_plan"]:
        if st_item["id"] == task_id:
            updated_plan_list.append({**st_item, "status": "completed", "result": analysis_output})
        else: updated_plan_list.append(st_item)
    return {"research_plan": updated_plan_list, "supervisor_next_directive": "PlannerSupervisor", "messages":[AIMessage(content=analysis_output, name="DataAnalyst")]}

def report_writer_node_coop(state: CooperativeResearchBotState):
    task_id = state["current_processing_sub_task_id"]
    task_object = next((st for st in state["research_plan"] if st["id"] == task_id), None)
    if not task_object: return {"supervisor_next_directive": "PlannerSupervisor"}
    task_description = task_object["description"]
    
    input_for_report = ""
    if task_object.get("input_data_source_ids"):
        for source_id in task_object["input_data_source_ids"]:
            source_task = next((t for t in state["research_plan"] if t["id"] == source_id), None)
            if source_task and source_task.get("result"):
                input_for_report += f"\n--- {source_task['description']} の結果 ---\n{source_task['result']}\n"
    if not input_for_report: input_for_report = "(レポート作成のための入力データなし)"

    print(f"\nReportWriter Agent: サブタスク「{task_description}」のレポートを作成します。入力(一部): {input_for_report[:100]}...")
    # final_report_doc = llm.invoke(ChatPromptTemplate.from_messages([SystemMessage(content="あなたはプロのレポートライターです。与えられた情報を元に、包括的で読みやすい報告書を作成してください。"), HumanMessage(content=f"以下の情報に基づいて、「{state['main_research_request']}」に関する報告書を作成してください。\n\n{input_for_report}")])).content
    final_report_doc = f"# 最終報告書: {state['main_research_request']}\n\n## 1. 調査概要\n{input_for_report[:150]}...\n\n## 2. 分析と洞察\n...（詳細な分析内容）...\n\n## 3. 結論と提言\n...（まとめと提言）...\n\n(この報告書は自動生成されました)"
    print(f"  -> 作成された報告書(一部): {final_report_doc[:150]}...")
    updated_plan_list = []
    for st_item in state["research_plan"]:
        if st_item["id"] == task_id:
            updated_plan_list.append({**st_item, "status": "completed", "result": final_report_doc})
        else: updated_plan_list.append(st_item)
    # 最終成果物を final_report_content にも設定する (Supervisorがこれを見るため)
    return {"research_plan": updated_plan_list, "final_report_content": final_report_doc, "supervisor_next_directive": "PlannerSupervisor", "messages":[AIMessage(content=final_report_doc, name="ReportWriter")]}

# --- ルーター関数 ---
def cooperative_workflow_router(state: CooperativeResearchBotState):
    next_agent_directive = state.get("supervisor_next_directive")
    print(f"\nCooperative Workflow Router: スーパーバイザーの次の指示 -> {next_agent_directive}")
    if next_agent_directive == "PlannerSupervisor": return "route_to_supervisor"
    if next_agent_directive == "WebSearcher": return "route_to_web_searcher"
    if next_agent_directive == "DataAnalyst": return "route_to_data_analyst"
    if next_agent_directive == "ReportWriter": return "route_to_report_writer"
    if next_agent_directive == "FINISH_WORKFLOW": return END
    print("  -> 不明な指示または終了。ENDへ")
    return END # デフォルトは終了

# --- グラフ構築 ---
workflow = StateGraph(CooperativeResearchBotState)

workflow.add_node("supervisor_cooperative", planner_supervisor_node)
workflow.add_node("web_searcher_cooperative", web_searcher_node_coop)
workflow.add_node("data_analyst_cooperative", data_analyst_node_coop)
workflow.add_node("report_writer_cooperative", report_writer_node_coop)

workflow.set_entry_point("supervisor_cooperative")

workflow.add_conditional_edges(
    "supervisor_cooperative", 
    cooperative_workflow_router,
    {
        "route_to_supervisor": "supervisor_cooperative", # プラン作成後など、再度スーパーバイザーに戻る場合
        "route_to_web_searcher": "web_searcher_cooperative",
        "route_to_data_analyst": "data_analyst_cooperative",
        "route_to_report_writer": "report_writer_cooperative",
        END: END
    }
)

# 各ワーカーエージェントは処理後、スーパーバイザーに戻る
workflow.add_edge("web_searcher_cooperative", "supervisor_cooperative")
workflow.add_edge("data_analyst_cooperative", "supervisor_cooperative")
workflow.add_edge("report_writer_cooperative", "supervisor_cooperative")

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
user_complex_request = "2024年のAI業界における主要な技術トレンドを調査し、それらがソフトウェア開発に与える影響を分析し、最終的な考察レポートを作成してください。"
print(f"\n--- 第4章まとめ 協調型リサーチボットテスト (リクエスト: '{user_complex_request}') ---")

inputs_cooperative_bot = {"messages": [HumanMessage(content=user_complex_request)], "main_research_request": user_complex_request}

print("\nストリーム出力:")
for i, event in enumerate(graph.stream(inputs_cooperative_bot, {"recursion_limit": 20})): # ステップ数と深さを考慮
    print(f"Event {i+1}: {event}")

final_state_coop_bot = graph.invoke(inputs_cooperative_bot, {"recursion_limit": 20})
print("\n最終状態の確認:")
print(f"  ユーザーリクエスト: {final_state_coop_bot['main_research_request']}")
print(f"  調査計画 ({len(final_state_coop_bot.get('research_plan',[]))}ステップ):")
for task_idx, task_item_final in enumerate(final_state_coop_bot.get('research_plan', [])):
    print(f"    {task_idx+1}. ID:{task_item_final['id']}, Desc:{task_item_final['description'][:40]}..., Assignee:{task_item_final['assigned_to']}, Status:{task_item_final['status']}, Result(part):{str(task_item_final.get('result'))[:50]}...")
print(f"  最終報告書(一部): {final_state_coop_bot.get('final_report_content', '(報告書なし)')[:250]}...")
print(f"  スーパーバイザーの最終指示: {final_state_coop_bot.get('supervisor_next_directive')}")

assert final_state_coop_bot.get('final_report_content') is not None and len(final_state_coop_bot.get('final_report_content', '')) > 0
assert final_state_coop_bot.get('supervisor_next_directive') == "FINISH_WORKFLOW"
assert all(task.get("status") == "completed" for task in final_state_coop_bot.get("research_plan", [])), "全てのサブタスクが完了ステータスではありません。"
print("\nアサーション成功！")
``````
</details>